Pada tugas pratikum ini Anda akan menggunakan data "Wisconsin Breast Cancer". Data tersebut terdiri dari 569 data yang digunakan untuk mendiagnonis jenis kanker Malignant (M) dan Benign (B). Tugas Anda adalah,

1. Pisahkan antara variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.
2. Lakukan proses encoding pada kolom "diagnosis".
3. Lakukan proses standardisasi pada semua kolom yang memiliki nilai numerik.
4. Lakukan proses seleksi fitur. Anda dapat menggunakan SelectKBest.
5. Lakukan proses pengujian dengan model Logistic Regression seperti pada praktikum 1.
6. Anda dapat menggunakan model pipeline untuk mempermudah perkejaan Anda.
7. Berdasarkan hasil analisa Anda, berapa jumlah fitur terbaik yang dapat digunakan? Apa saja fitur tersebut?

In [42]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2, SelectKBest, RFE
from sklearn.metrics import accuracy_score, classification_report

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [35]:
df = pd.read_csv('wbc.csv')
df.head(10)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,NaN
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,NaN
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,NaN


In [18]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [19]:
df.iloc[:,5:7] #mengambil data kolom 5 sampai 7

,area_mean,smoothness_mean
0,1001.0,0.11840
1,1326.0,0.08474
2,1203.0,0.10960
3,386.1,0.14250
4,1297.0,0.10030
...,...,...
564,1479.0,0.11100
565,1261.0,0.09780
566,858.1,0.08455
567,1265.0,0.11780


In [20]:
#1. Pisahkan variabel yang bisa & tidak bisa digunakan

# Kolom yang tidak berguna
not_usable_cols = ["id", "Unnamed: 32"]

# Buang kolom yang tidak diperlukan
df_use = df.drop(columns=not_usable_cols)

print("Kolom yang digunakan : ", df_use.columns.tolist())
print("Kolom yang tidak digunakan : ", not_usable_cols)


Kolom yang digunakan :  ['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
Kolom yang tidak digunakan :  ['id', 'Unnamed: 32']


In [21]:
#2. Encoding kolom diagnosis (M → 1, B → 0)

# Encoding diagnosis
label_encoder = LabelEncoder()
df_use["diagnosis"] = label_encoder.fit_transform(df_use["diagnosis"])

# Cek hasil encoding
print(df_use["diagnosis"].value_counts())

diagnosis
0    357
1    212
Name: count, dtype: int64


In [22]:
#3. Standardisasi pada semua kolom numerik

# Ambil semua kolom numerik (selain diagnosis)
numeric_cols = df_use.drop(columns=["diagnosis"]).select_dtypes(include=[np.number]).columns

# Standardisasi
scaler = StandardScaler()
df_use[numeric_cols] = scaler.fit_transform(df_use[numeric_cols])

df_use.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,1,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100


In [39]:
# 4. Seleksi fitur dengan SelectKBest

X = df_use.drop(columns=["diagnosis"])
y = df_use["diagnosis"]

# Gunakan SelectKBest (ANOVA F-test)
selector = SelectKBest(score_func=f_classif, k="all")  # ambil semua dulu
X_new = selector.fit_transform(X, y)

# Tampilkan skor tiap fitur
scores = pd.DataFrame({
    "Feature": X.columns,
    "Score": selector.scores_
}).sort_values(by="Score", ascending=False)

print("\n Skor tiap fitur (semakin besar semakin penting):")
print(scores.head(15))  # tampilkan 15 teratas


 Skor tiap fitur (semakin besar semakin penting):
                 Feature       Score
27  concave points_worst  964.385393
22       perimeter_worst  897.944219
7    concave points_mean  861.676020
20          radius_worst  860.781707
2         perimeter_mean  697.235272
23            area_worst  661.600206
0            radius_mean  646.981021
3              area_mean  573.060747
6         concavity_mean  533.793126
26       concavity_worst  436.691939
5       compactness_mean  313.233079
25     compactness_worst  304.341063
10             radius_se  268.840327
12          perimeter_se  253.897392
13               area_se  243.651586


In [40]:
# 5. Pengujian dengan Logistic Regression (tanpa pipeline)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Prediksi
y_pred = log_reg.predict(X_test)

print("\n Akurasi Logistic Regression:", accuracy_score(y_test, y_pred))
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print("\n Confusion Matrix:\n", confusion_matrix(y_test, y_pred))



 Akurasi Logistic Regression: 0.9649122807017544

 Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97        72
           1       0.97      0.93      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114


 Confusion Matrix:
 [[71  1]
 [ 3 39]]


In [44]:
# 6. Pipeline: Standardisasi + Seleksi Fitur + Logistic Regression

results = {}
for k in range(1, X.shape[1] + 1):
    pipe = Pipeline([
        ('select', SelectKBest(score_func=f_classif, k=k)),
        ('model', LogisticRegression(max_iter=1000))
    ])
    scores = cross_val_score(pipe, X, y, cv=5)
    results[k] = scores.mean()

best_k = max(results, key=results.get)
print("\n Jumlah fitur terbaik:", best_k)
print("Akurasi terbaik (CV 5-fold):", results[best_k])



 Jumlah fitur terbaik: 27
Akurasi terbaik (CV 5-fold): 0.9806862288464524


In [46]:
# 7. Fitur Terpilih

final_selector = SelectKBest(score_func=f_classif, k=best_k)
X_new = final_selector.fit_transform(X, y)
selected_features = X.columns[final_selector.get_support()]

print("\n Fitur terbaik yang dipilih:")
print(list(selected_features))


 Fitur terbaik yang dipilih:
['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'radius_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
